# **1-1. 통합 테스트**

## **1.환경설정**

### (1) 경로 설정

로컬 경로 설정

In [1]:
path = './'

### (2) 라이브러리

In [2]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import torch
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 더 필요한 라이브러리 추가 -------------
import emergency as em

## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

### (1) open ai key 등록

In [17]:
openai.api_key = em.load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

### (2) audio to text

In [9]:
audio_path = path + 'audio/'
filename = 'audio1.mp3'
result = em.audio_to_text(audio_path, filename)
result

'지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.\n'

### (3) text summary

In [11]:
result = em.text_summary(result)
result

'39세의 아버지가 계단에서 넘어져 머리에서 피가 나며 숨을 쉬고 있습니다. 응급상황으로 응급실로 이동해야 합니다., 아버지, 머리 출혈, 응급실'

### (4) 응급실 등급분류

In [12]:
# 모델, 토크나이저 로드
save_directory = path + "fine_tuned_bert"
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [13]:
predicted_class, probabilities = em.predict(result, model, tokenizer)

print(f"예측된 클래스: {predicted_class}")
print(f"예측된 클래스 이름: {predicted_class+1}등급")
print(f"클래스별 확률: {probabilities}")

예측된 클래스: 1
예측된 클래스 이름: 2등급
클래스별 확률: tensor([[0.1670, 0.3206, 0.1676, 0.1936, 0.1512]])


### (5) 응급실추천

In [14]:
emergency = pd.read_csv(path + '응급실 정보.csv')

map_key = em.load_file(path + 'map_key.txt')
map_key = json.loads(map_key)
c_id, c_key = map_key['c_id'], map_key['c_key']

In [15]:
predicted_class = 2
start_lat, start_lng = 35.548238, 129.307011

if predicted_class <= 2 :
    result = em.recommend_hospital3(path, emergency, start_lat, start_lng, c_id, c_key)

result

응답 데이터에서 예상되는 키를 찾을 수 없음: 'route'


,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,거리
299,의료법인동강의료재단동강병원,울산광역시 중구 태화로 239 (태화동),지역응급의료센터,052-241-1114,052-241-1190,35.553410,129.301943,2.032
367,의료법인정안의료재단중앙병원,"울산광역시 남구 문수로 472, 중앙병원 (신정동)",지역응급의료센터,052-226-1100,052-226-1119,35.532088,129.304840,2.125
252,울산병원,울산광역시 남구 월평로171번길 13 (신정동),지역응급의료센터,052-259-5000,052-259-5119,35.546217,129.323100,2.390


## **3. 통합 테스트**

* 세부사항
    * [2. 단위 테스트] 의 내용을 순차적으로 실행하며 결과를 확인합니다.

In [16]:
# 1. 준비 작업 ---------------------------------------------
openai.api_key = em.load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

map_key = em.load_file(path + 'map_key.txt')
map_key = json.loads(map_key)
c_id, c_key = map_key['c_id'], map_key['c_key']

emergency = pd.read_csv(path + '응급실 정보.csv')

# 모델, 토크나이저 로드
save_directory = path + "fine_tuned_bert"
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)


# 2. 데이터 처리(pipeline) ---------------------------------
# input
audio_path = path + 'audio/'
filename = 'audio1.mp3'

# 처리
result = em.audio_to_text(audio_path, filename)
result = em.text_summary(result)
predicted_class, _ = em.predict(result, model, tokenizer)

# 테스트 input --------------------
predicted_class = 2  # 테스트용
start_lat, start_lng = 35.548238, 129.307011
# ---------------------------------

if predicted_class <= 2 :
    result = em.recommend_hospital3(path, emergency, start_lat, start_lng, c_id, c_key)
    display(result)
else :
    print('개인 건강관리')

응답 데이터에서 예상되는 키를 찾을 수 없음: 'route'


,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,거리
299,의료법인동강의료재단동강병원,울산광역시 중구 태화로 239 (태화동),지역응급의료센터,052-241-1114,052-241-1190,35.553410,129.301943,2.032
367,의료법인정안의료재단중앙병원,"울산광역시 남구 문수로 472, 중앙병원 (신정동)",지역응급의료센터,052-226-1100,052-226-1119,35.532088,129.304840,2.125
252,울산병원,울산광역시 남구 월평로171번길 13 (신정동),지역응급의료센터,052-259-5000,052-259-5119,35.546217,129.323100,2.390
